In [28]:
import regex as re
import json

def pretty_print(dict):
    print(json.dumps(dict, indent=4))

---
# Songs
---

In [29]:
# Twinkle, Twinkle, Little Star

twinkle = {
    'title':"Twinkle, Twinkle, Little Star",
    'pitches':"CCGGAAG FFEEDDC GGFFEED GGFFEED CCGGAAG FFEEDDC"
#     'pitches':"CCGGAAGFFEEDDCGGFFEEDGGFFEEDCCGGAAGFFEEDDC"
}

In [30]:
# Bah, Bah, Black Sheep

bah = {
    'title':"Bah, Bah, Black Sheep",
    'pitches':"CCGGAAAAG FFEEDDC GGGFFFEEED GGGFFFFEEED CCGGAAAAG FFEEDDC"
}

In [31]:
# The ABC song

abc = {
    'title':"The ABC Song",
    'pitches':"CCGGAAG FFEEDDDDC GGFEED GGGFEED CCGGAAG FFEEDDC"
}

In [32]:
# Frere Jacques

frere = {
    'title':"Frere Jacques",
    'pitches':"CDEC CDEC EFG EFG GAGFEC GAGFEC CGC CGC"
}

In [33]:
# Three Blind Mice

mice = {
    'title':"Three Blind Mice",
    'pitches':"EDC EDC GFFE GFFE GCCBABCGG GCCCBABCGG GGCCBABCGGG FEDC"
}

In [34]:
# Hot Cross Buns

buns = {
    'title':"Hot Cross Buns",
    'pitches': "EDC EDC CCCC DDDD EDC"
}

In [35]:
# Row, Row, Row Your Boat

row = {
    'title':"Row, Row, Row Your Boat",
    'pitches':"CCCDE EDEFG CCCGGGEEECCC GFEDC"
}

In [36]:
# Mary Had a Little Lamb

lamb = {
    'title':"Mary Had a Little Lamb",
    'pitches':"EDCDEEE DDD EEE EDCDEEE EDDEDC"
}

In [37]:
#create a dictionary for storing titles, pitches, and patterns
all_songs = {}
for song in [twinkle,bah,abc,frere,mice,buns,row,lamb]:
    all_songs[song['title']] = {'pitches':song['pitches'].replace(' ',''),'patterns':[]}

---
# Regex Searches
---

In [38]:
#set search parameters
min_length = 4
min_occur = 4

for song in all_songs:
    pitches = all_songs[song]['pitches']

    pat = r'(.{%d,})(?:.*\1){%d,}' % (min_length, min_occur-1)

    pitch_patterns = re.findall(pat,pitches,overlapped=True)
   
    try:
        max_length = len(sorted(pitch_patterns, key=len, reverse=True)[0])

        for length in range(max_length,min_length-1,-1):
            pat = r'(.{%d})(?:.*\1){%d,}' % (length, min_occur-1)
            pitch_patterns = re.findall(pat,pitches,overlapped=True)
            for pattern in pitch_patterns:
                if pattern not in all_songs[song]['patterns']:
                    all_songs[song]['patterns'].append(pattern)
                else:
                    continue
    except:
        continue  

In [39]:
pretty_print(all_songs)

{
    "Twinkle, Twinkle, Little Star": {
        "pitches": "CCGGAAGFFEEDDCGGFFEEDGGFFEEDCCGGAAGFFEEDDC",
        "patterns": [
            "GFFEED",
            "GFFEE",
            "FFEED",
            "GFFE",
            "FFEE",
            "FEED"
        ]
    },
    "Bah, Bah, Black Sheep": {
        "pitches": "CCGGAAAAGFFEEDDCGGGFFFEEEDGGGFFFFEEEDCCGGAAAAGFFEEDDC",
        "patterns": [
            "FFEE"
        ]
    },
    "The ABC Song": {
        "pitches": "CCGGAAGFFEEDDDDCGGFEEDGGGFEEDCCGGAAGFFEEDDC",
        "patterns": [
            "FEED"
        ]
    },
    "Frere Jacques": {
        "pitches": "CDECCDECEFGEFGGAGFECGAGFECCGCCGC",
        "patterns": []
    },
    "Three Blind Mice": {
        "pitches": "EDCEDCGFFEGFFEGCCBABCGGGCCCBABCGGGGCCBABCGGGFEDC",
        "patterns": []
    },
    "Hot Cross Buns": {
        "pitches": "EDCEDCCCCCDDDDEDC",
        "patterns": []
    },
    "Row, Row, Row Your Boat": {
        "pitches": "CCCDEEDEFGCCCGGGEEECCCGFEDC",
        "

---
# SQLite3
---

In [40]:
import sqlite3
conn = sqlite3.connect('childrens_songs.db')
c = conn.cursor()

## Create tables

## Insert values into tables

### Insert multiple rows as list of tuples

### Insert single rows by iterating through list

### Pre-optimized code

### Somewhat-optimized code

In [41]:
# for each song
for title in all_songs:
    
    # if song not in song table, store song in song table
    if c.execute('''SELECT COUNT(song_title) FROM song WHERE song_title=(?)''', (title,)).fetchone()[0] == 0:
        c.execute('''INSERT INTO song(song_title) VALUES (?)''',(title,))
    
    # get song id from song table
    song_id = c.execute('''SELECT song_id FROM song WHERE song_title=?''', (title,)).fetchone()[0]

    # for each pattern
    for pattern in all_songs[title]['patterns']:
        
        # if pattern not in pattern table, store pattern in pattern table
        if c.execute('''SELECT COUNT(pattern_string) FROM pattern WHERE pattern_string=(?)''', (pattern,)).fetchone()[0] == 0:
            c.execute('''INSERT INTO pattern(pattern_string) VALUES (?)''',(pattern,))

        # get pattern id from pattern table
        pattern_id = c.execute('''SELECT pattern_id FROM pattern WHERE pattern_string=?''', (pattern,)).fetchone()[0]
        
        # store song id, pattern id in song_pattern table
        if c.execute('''SELECT COUNT(song_id) FROM song_pattern WHERE song_id=(?) AND pattern_id=(?)''', (song_id,pattern_id)).fetchone()[0] == 0:
            c.execute('''INSERT INTO song_pattern(song_id, pattern_id) VALUES (?,?)''',(song_id,pattern_id))

## Query all the tables!

### Check for correct values in tables

In [42]:
for row in c.execute('''SELECT * FROM song;'''):
    print(row)

(1, 'Twinkle, Twinkle, Little Star')
(2, 'Bah, Bah, Black Sheep')
(3, 'The ABC Song')
(4, 'Frere Jacques')
(5, 'Three Blind Mice')
(6, 'Hot Cross Buns')
(7, 'Row, Row, Row Your Boat')
(8, 'Mary Had a Little Lamb')


In [43]:
for row in c.execute('''SELECT * FROM pattern;'''):
    print(row)

(1, 'GFFEED')
(2, 'GFFEE')
(3, 'FFEED')
(4, 'GFFE')
(5, 'FFEE')
(6, 'FEED')


In [44]:
for row in c.execute('''SELECT * FROM song_pattern;'''):
    print(row)

(1, 1)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
(1, 6)
(2, 5)
(3, 6)


### SQLite3 practice queries

In [1]:
%%HTML
<img src="https://i.imgflip.com/3us2ix.jpg" title="made at imgflip.com"/>

### "Now we're getting somewhere" queries

In [46]:
# return all of the songs and patterns (group patterns with GROUP_CONCAT() function)

query = '''
    SELECT song.song_id, song_title, pattern.pattern_id, GROUP_CONCAT(pattern_string)
    FROM (song
    LEFT JOIN song_pattern ON song.song_id=song_pattern.song_id)
    LEFT JOIN pattern ON pattern.pattern_id=song_pattern.pattern_id
    GROUP BY song.song_id;'''
for row in c.execute(query):
    print(row)

(1, 'Twinkle, Twinkle, Little Star', 1, 'GFFEED,GFFEE,FFEED,GFFE,FFEE,FEED')
(2, 'Bah, Bah, Black Sheep', 5, 'FFEE')
(3, 'The ABC Song', 6, 'FEED')
(4, 'Frere Jacques', None, None)
(5, 'Three Blind Mice', None, None)
(6, 'Hot Cross Buns', None, None)
(7, 'Row, Row, Row Your Boat', None, None)
(8, 'Mary Had a Little Lamb', None, None)


In [47]:
# return song title and how many patterns that song contains, if song contains pattern (determined by join type)

query = '''
    SELECT song_title, count(pattern_string)
    FROM (song
    JOIN song_pattern ON song.song_id=song_pattern.song_id)
    JOIN pattern ON pattern.pattern_id=song_pattern.pattern_id
    GROUP BY song.song_id;'''
for row in c.execute(query):
    print(row)

('Twinkle, Twinkle, Little Star', 6)
('Bah, Bah, Black Sheep', 1)
('The ABC Song', 1)


In [48]:
# return all songs that have more than one pattern

query = '''
    SELECT song_title, GROUP_CONCAT(pattern_string)
    FROM (song
    JOIN song_pattern ON song.song_id=song_pattern.song_id)
    JOIN pattern ON pattern.pattern_id=song_pattern.pattern_id
    GROUP BY song.song_id
    HAVING COUNT(pattern_string) > 1;'''

for row in c.execute(query):
    print(row)

('Twinkle, Twinkle, Little Star', 'GFFEED,GFFEE,FFEED,GFFE,FFEE,FEED')


In [49]:
# return all patterns that occur in more than one song, group concat the song titles

query = '''
    SELECT pattern_string, GROUP_CONCAT(song_title, ' | ')
    FROM (song
    JOIN song_pattern ON song.song_id=song_pattern.song_id)
    JOIN pattern ON pattern.pattern_id=song_pattern.pattern_id
    GROUP BY pattern.pattern_string
    HAVING COUNT(pattern_string) > 1;'''

for row in c.execute(query):
    print(row)

('FEED', 'Twinkle, Twinkle, Little Star | The ABC Song')
('FFEE', 'Twinkle, Twinkle, Little Star | Bah, Bah, Black Sheep')


---
# Random
---

In [4]:
import os,regex as re

In [8]:
files = os.listdir("/Users/BrandonBel/Desktop/MusicXML_scores/csv/")
for i, f in enumerate(sorted(files),1):
    try:
        m = re.match(r'(\w*?)-(.*).csv',f)
        print(i, m.group(1),m.group(2))
    except:
        print("Cannot convert: %s" % f)
        

Cannot convert: .DS_Store
2 Beethoven 01_variation
3 Beethoven 02_variation
4 Beethoven 03_variation
5 Beethoven 04_variation
6 Beethoven 05_variation
7 Beethoven 06_variation
8 Beethoven 07_variation
9 Beethoven 08_variation
10 Beethoven 09_variation
11 Beethoven 0_diabelli
12 Beethoven 1-Allegro
13 Beethoven 10_variation
14 Beethoven 11_variation
15 Beethoven 12_variation
16 Beethoven 13_variation
17 Beethoven 14_variation
18 Beethoven 15_variation
19 Beethoven 16_variation
20 Beethoven 18_variation
21 Beethoven 19_variation
22 Beethoven 1_Horn1
23 Beethoven 1_Satz, Allegro
24 Beethoven 2-Allegretto
25 Beethoven 20_variation
26 Beethoven 21_variation
27 Beethoven 22_variation
28 Beethoven 23_variation
29 Beethoven 24_variation
30 Beethoven 25_variation
31 Beethoven 26_variation
32 Beethoven 27_variation
33 Beethoven 28_variation
34 Beethoven 29_variation
35 Beethoven 2_Horn2
36 Beethoven 2_Satz, Andante
37 Beethoven 3-Rondo
38 Beethoven 30_variation
39 Beethoven 31_variation
40 Beeth